In [226]:
# Parameters
GROUP_ID = '43'
ALGORITHM = 'bt' # bt | fc | ac3 | sa | ga
PUZZLE_TYPE = 'extreme' # easy | medium | hard | extreme
PUZZLE_PATH = 'C:\\Users\\Asus\\Downloads\\AiSudokuSolver-main\\AiSudokuSolver-main\\puzzles\\easy\\Easy-P2.txt'

In [227]:
from io_1 import read_puzzle, write_puzzle, build_output_filename
import numpy as np
import hashlib
import random
from random import choice
# Deterministic seed from parameters
_seed = int(hashlib.md5(f"{GROUP_ID}:{ALGORITHM}:{PUZZLE_PATH}".encode()).hexdigest()[:8], 16)

In [228]:
def flag_Initial(grid):
    flag = np.zeros((9,9), dtype=int)
    for i in range (0,9):
        for j in range (0,9):
            if grid[i][j] != 0:
                flag[i,j] = 1
    return flag

In [229]:
def mutate(gridtemp , flag):
    temporary = gridtemp.copy()
    while True:
        row1 =  random.randint(0,8)
        col1 =  random.randint(0,8)
        while True:
            r = random.randint(0,8)
            c = random.randint(0,8)
            if (r,c) != (row1,col1):
                row2 = r
                col2 = c
                break
        if flag[row1][col1] != 1 and flag[row2][col2] != 1:
            if random.random() < 0.15:
                temp = temporary[row1][col1]
                temporary[row1][col1] = temporary[row2][col2]
                temporary[row2][col2] = temp
            return temporary

In [230]:
def fitness(grid):
    ErrorCount = 0
    total = 0
    for i in range(0,9):
        ErrorCount = (9 - len(np.unique(grid[:,i]))) + (9 - len(np.unique(grid[i,:])))
        total = total + ErrorCount
    return total

In [231]:
def population_initial(grid, flag):
    temp = grid.copy()
    for i in range(0,9):
        for j in range(0,9):
            if flag[i][j] != 1:
                temp[i][j] = random.randint(1,9)
    return temp

In [232]:
def population_initial_update(grid):
    tempgrid = grid.copy()
    blocks = []
    for r in range(3):
        for c in range(3):
            block = []
            for i in range(3):
                for j in range(3):
                    block.append(tempgrid[3*r + i][3*c + j])
            blocks.append(block)
    for block in blocks:
        for j in range(0,9):
            if block[j] == 0:
                    block[j] = choice([i for i in range(1,10) if i not in block])
    randomgrid = []
    for block_row in range(3):       
        for row_in_block in range(3): 
            row = []
            for block_col in range(3):  
                block_index = block_row * 3 + block_col
                row.extend(blocks[block_index][row_in_block*3 : (row_in_block+1)*3])
            randomgrid.append(row)
    randomgrid = np.array(randomgrid)
    return randomgrid

In [233]:
def tournament_selection(population, fitnesses):
    tournament_size = 3
    selected_indices = random.sample(range(len(population)), tournament_size)
    selected = [population[i] for i in selected_indices]
    selected_fitnesses = [fitnesses[i] for i in selected_indices]
    winner_index = selected_indices[selected_fitnesses.index(min(selected_fitnesses))]
    return population[winner_index]

In [234]:
def crossover(parent1, parent2, flag):
    child = parent1.copy()
    row = random.randint(0,8)
    col = random.randint(0,8)
    for i in range(row,9):
        for j in range(col,9):
            if flag[i][j] != 1:
                child[i][j] = parent2[i][j]
    return child

In [235]:
initialgrid = read_puzzle(PUZZLE_PATH)
# print(initialgrid)
flag = flag_Initial(initialgrid)
grid = initialgrid.copy()
first_gen = []
population_size = 100
for p in range (0, population_size):
    temp = population_initial_update(grid)
    temp = np.array(temp)
    first_gen.append(temp.copy())
current_gen = first_gen
current_gen = np.array(current_gen)
for generation in range(1000):
    print(f"Generation {generation+1}")
    fitness_list = []
    next_gen = []
    for individual in current_gen:
        fitness_list.append(fitness(individual))    
    average = sum(fitness_list)/len(fitness_list)
    print("Average Fitness: ", average)
    print("Best Fitness: ", min(fitness_list))
    if 0 in fitness_list:
        index = fitness_list.index(0)
        print("Solution Found")
        print(first_gen[index])
        break
    for i in range(50):
        parent1 = tournament_selection(current_gen, fitness_list)
        parent2 = tournament_selection(current_gen, fitness_list)
        child1 = mutate(crossover(parent1, parent2, flag),flag)
        child2 = mutate(crossover(parent2, parent1, flag),flag)
        next_gen.append(child1)
        next_gen.append(child2)    
        # print("%d pairs created\n"%(i+1))
        # print("NextGen Size: ",len(next_gen))
    current_gen = next_gen
    print(len(current_gen))
    
    



Generation 1
Average Fitness:  36.94
Best Fitness:  28
100
Generation 2
Average Fitness:  34.52
Best Fitness:  26
100
Generation 3
Average Fitness:  32.48
Best Fitness:  26
100
Generation 4
Average Fitness:  30.83
Best Fitness:  24
100
Generation 5
Average Fitness:  28.96
Best Fitness:  21
100
Generation 6
Average Fitness:  27.2
Best Fitness:  20
100
Generation 7
Average Fitness:  26.13
Best Fitness:  22
100
Generation 8
Average Fitness:  25.28
Best Fitness:  20
100
Generation 9
Average Fitness:  23.86
Best Fitness:  19
100
Generation 10
Average Fitness:  22.7
Best Fitness:  18
100
Generation 11
Average Fitness:  21.45
Best Fitness:  17
100
Generation 12
Average Fitness:  20.72
Best Fitness:  17
100
Generation 13
Average Fitness:  20.54
Best Fitness:  17
100
Generation 14
Average Fitness:  19.52
Best Fitness:  16
100
Generation 15
Average Fitness:  18.88
Best Fitness:  16
100
Generation 16
Average Fitness:  18.33
Best Fitness:  15
100
Generation 17
Average Fitness:  18.05
Best Fitness: